In [ ]:
from glob import glob
import os
import os.path as osp

import pandas as pd

In [ ]:
FOLDER = './drive/MyDrive/Colab Notebooks/market caps/'
files = glob(FOLDER + '*.csv')

In [ ]:
BTC_FILE = 'bitcoin_2011-01-01_2025-01-01.csv'

btc_ind = files.remove(FOLDER + BTC_FILE)

In [ ]:
# start with BTC
full_dt = pd.read_csv(FOLDER + BTC_FILE)
full_dt = full_dt[['Start', 'Market Cap']]
full_dt.rename(columns={
        'Start': 'Date',
        'Market Cap': 'bitcoin',
    },
    inplace=True,
)


In [ ]:
# join others
for file_ in files:
    _, tail = osp.split(file_)
    coin_name = tail.split('_')[0]

    dt = pd.read_csv(file_)
    dt = dt[['Start', 'Market Cap']]
    dt.rename(columns={
            'Start': 'Date',
            'Market Cap': coin_name,
        },
        inplace=True,
    )

    full_dt = pd.merge(full_dt, dt, how='outer', on='Date')

In [ ]:
# add other info

coins = list(full_dt.columns)
coins.remove("Date")

# total market cap (of top 100)
full_dt.insert(
    loc=1,
    column="total market cap",
    value=full_dt.sum(axis=1, numeric_only=True),
)

# ranks
ranks = full_dt[coins].rank(axis=1, ascending=False)
full_dt = pd.merge(full_dt, ranks, left_index=True, right_index=True, suffixes=('', '_rank'))

In [ ]:
full_dt.to_excel(FOLDER + 'joined.xlsx')

In [ ]:
full_dt